In [ ]:
from IPython.display import SVG
import svgwrite
import numpy as np
import os
import random
import pandas as pd
from random import randint
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from torch.distributions.categorical import Categorical
from torch.distributions.normal import Normal
from torch.distributions.multivariate_normal import MultivariateNormal
import warnings
import cmath
from svgpathtools import *
import math

In [ ]:
from PIL import Image, ImageDraw
from PIL.ImagePalette import ImagePalette

In [ ]:
from cairosvg.parser import Tree
import cairo

In [ ]:
import cairosvg

In [ ]:
from deepsvg.svglib.utils import to_gif

In [ ]:
import deepsvg.svglib.svg

In [ ]:
#import pytorch_lightning as pl

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
rotation_matrix = lambda angle: torch.tensor([[np.cos(angle),-np.sin(angle)],[np.sin(angle),np.cos(angle)]])
flip_x_matrix = torch.tensor([[-1.0,0.0],[0.0,1.0]])
flip_y_matrix = torch.tensor([[1.0,0.0],[0.0,-1.0]])
flip_o_matrix = torch.tensor([[-1.0,0.0],[0.0,-1.0]])
flip_xy_matrix = torch.tensor([[0.0,1.0],[1.0,0.0]])
scale_x_matrix = lambda scale_x: torch.tensor([[scale_x,0.0],[0,1.0]]) #scale x
scale_y_matrix = lambda scale_y: torch.tensor([[1,0],[0,scale_y]]) # scale y
scale_xy_matrix = lambda scale_x,scale_y: torch.tensor([[scale_x,0],[0,scale_y]]) #scale xy
skew_x_matrix = lambda scale_x: torch.tensor([[1,scale_x],[0,1]]) #skew x
skew_y_matrix = lambda scale_y: torch.tensor([[1,0],[scale_y,1]]) #skew y 
skew_xy_matrix = lambda scale_x,scale_y: torch.tensor([[1,scale_x],[scale_y,1]]) #skew xy

In [ ]:
#precent = 100
#variance =0.6
#print(f'The variance is different by {precent}% and {"yes" if variance >0.5 else "no"}')

In [ ]:
transformed_tensor = lambda tensor,matrix: torch.mm(tensor,matrix)

In [ ]:
paths, attributes, svg_attributes = svg2paths2('Minisample/leaves_base_2.svg')

In [ ]:
attributes[0]['style']

In [ ]:
svg_attributes

In [ ]:
wsvg(paths[0],filename='variable_01.svg')

In [ ]:
len(paths)

In [ ]:
def perform_transformations(points):
    figures = [torch.unsqueeze(points,0)]
    #figures.append(torch.unsqueeze(transformed_tensor(points,flip_x_matrix),0))
    #figures.append(torch.unsqueeze(transformed_tensor(points,flip_y_matrix),0))
    #figures.append(torch.unsqueeze(transformed_tensor(points,flip_o_matrix),0))
    #figures.append(torch.unsqueeze(transformed_tensor(points,flip_xy_matrix),0))
    return figures

In [ ]:
n_seg = 584 #296#152 #produces equally sized vectors
ts = np.array(range(n_seg+1))*1/n_seg
figures = []
for path in paths:
    wsvg(path,filename='variable_01.svg')
    plant = deepsvg.svglib.svg.SVG.load_svg('variable_01.svg').normalize().canonicalize()
    plant.save_svg('variable_01.svg')
    path, _, _ = svg2paths2('variable_01.svg')
    path = path[0]
    points = torch.tensor([[path.point(t).real,path.point(t).imag] for t in ts]).float()
    figures += perform_transformations(points)
training_data = torch.cat(figures).to(cuda0) #dtype=torch.float,device=cuda0
training_data = training_data[:,1:,:] - training_data[:,:-1,:] #OPTIONAL
training_data = training_data.permute((0,2,1))
training_data.shape

In [ ]:
inputs = []
for path in paths:
    wsvg(path,filename='variable_01.svg')
    plant = deepsvg.svglib.svg.SVG.load_svg('variable_01.svg').normalize().canonicalize()
    plant.save_svg('variable_01.svg')
    path, _, _ = svg2paths2('variable_01.svg')
    path = path[0]
    end_points = [path.start]
    for segment in path:
        end_points.append(segment.end)
    end_points = np.array(end_points)
        #print(end_points[np.where(np.sqrt((end_points[1:]-end_points[:-1]).real**2 + (end_points[1:]-end_points[:-1]).imag**2)<=0.5)])
    key_points = torch.tensor(list(map(complex,end_points[np.where(np.sqrt((end_points[1:]-end_points[:-1]).real**2 + (end_points[1:]-end_points[:-1]).imag**2)<=1)]))) #.float())
    inputs.append(torch.unsqueeze(torch.cat((key_points,torch.zeros(10-len(key_points)))),0))
input_data = torch.cat(inputs).to(cuda0)
print(input_data.shape,training_data.shape)

In [ ]:
input_data_split = torch.cat((torch.unsqueeze(input_data.real.float(),1),torch.unsqueeze(input_data.imag.float(),1)),dim=1)

In [ ]:
input_data_split = input_data_split/torch.max(input_data_split)

In [ ]:
input_data_split = input_data_split[:,:,1:]-input_data_split[:,:,:-1]

In [ ]:
input_data_split = input_data_split.permute((0,2,1))

In [ ]:
x_batches = torch.split(training_data,32)

In [ ]:
guide_batches = torch.split(input_data_split,32)

In [ ]:
class CNNArtist(nn.Module):

    def __init__(self): #, dropout=0.1
        super(CNNArtist, self).__init__()
    
        self.encoder = nn.Sequential( # We may add pooling as well, though works perfectly without it 
            nn.Conv1d(2, 2, kernel_size=5,stride=3), #ch: 6, stride=2 
            nn.LeakyReLU(True), 
            nn.Conv1d(2, 2, kernel_size=5,stride=3), #ch: 16 play with those hyperparameters
            nn.LeakyReLU(True))
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=3),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=3),
            nn.LeakyReLU(True),
            nn.Linear(584,584)) 
        self.fc_mu = nn.Linear(64, 64)
        self.fc_var = nn.Linear(64, 64)
        self.log_scale = nn.Parameter(torch.Tensor([0.0]))

    def forward(self, x):
        x_encoded = self.encoder(x)
        mu, log_var = self.fc_mu(x_encoded), self.fc_var(x_encoded)
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        z = q.rsample()
        #print(z.shape)
        x_hat = self.decoder(z)
        return x_hat,z,mu,std

In [ ]:
class MapToLatent(nn.Module):
    def __init__(self): #, dropout=0.1
        super(MapToLatent, self).__init__()
    
        self.encoder_mu = nn.Sequential(
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=1),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=2),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 1, kernel_size=4,stride=2),
            nn.LeakyReLU(True),
            nn.Linear(64,64)) 
        
        self.encoder_std = nn.Sequential(
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=1),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=2),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 1, kernel_size=4,stride=2),
            nn.LeakyReLU(True),
            nn.Linear(64,64)) 
    def forward(self, x):
        return self.encoder_mu(x),torch.exp(self.encoder_std(x))

In [ ]:
class MapToLatent_2(nn.Module):
    def __init__(self): #, dropout=0.1
        super(MapToLatent_2, self).__init__()
    
        self.encoder = nn.Sequential(
            nn.ConvTranspose1d(2, 2, kernel_size=6,stride=1),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 2, kernel_size=5,stride=2),
            nn.LeakyReLU(True),
            nn.ConvTranspose1d(2, 1, kernel_size=4,stride=2),
            nn.LeakyReLU(True),
            nn.Linear(64,64)) 
        
    def forward(self, x):
        return self.encoder(x)

In [ ]:
class MapToLatent_3(nn.Module):
    def __init__(self): #, dropout=0.1
        super(MapToLatent_3, self).__init__()
    
        self.encoder = nn.LSTM(2,128,num_layers=2, dropout=0.1, batch_first=True)    
        self.output_mu = nn.Linear(128,64)
        self.output_std = nn.Linear(128,64) 
        #self.encoder_layer = nn.TransformerEncoderLayer(d_model=2, dim_feedforward = 128, nhead=2)
        #self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        
    def forward(self, x): 
        _,(encoded,_) = self.encoder(x)
        #encoded = self.transformer_encoder(x.permute(1,0,2)) #.permute(2,0,1)
        #return self.output_mu(encoded),torch.exp(self.output_std(encoded))
        return self.output_mu(encoded)

In [ ]:
def gaussian_likelihood(x_hat, logscale, x):
    scale = torch.exp(logscale)
    mean = x_hat
    dist = torch.distributions.Normal(mean, scale)

    # measure prob of seeing image under p(x|z)
    log_pxz = dist.log_prob(x)
    return log_pxz.sum(dim=(1, 2))

In [ ]:
def kl_divergence(z, mu, std):
    # --------------------------
    # Monte carlo KL divergence
    # --------------------------
    # 1. define the first two probabilities (in this case Normal for both)
    p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
    q = torch.distributions.Normal(mu, std)

    # 2. get the probabilities from the equation
    log_qzx = q.log_prob(z)
    log_pz = p.log_prob(z)

    # kl
    kl = (log_qzx - log_pz)
    kl = kl.sum(dim=(1,2))
    return kl

In [ ]:
def kl_divergence_mapping(z, mu_real, std_real,mu,std):
    # --------------------------
    # Monte carlo KL divergence
    # --------------------------
    # 1. define the first two probabilities (in this case Normal for both)
    p = torch.distributions.Normal(mu_real, std_real)
    q = torch.distributions.Normal(mu, std)
    

    # 2. get the probabilities from the equation
    log_qzx = q.log_prob(z)
    log_pz = p.log_prob(z)
    
    # kl
    kl = (log_qzx - log_pz)
    kl = kl.sum(dim=(1,2))
    return kl

In [ ]:
#model = CNNArtist().to(cuda0)
#model(x_batches[0])

In [ ]:
model = CNNArtist().to(cuda0)
mse_loss = nn.MSELoss()
#lr = 0.5 # learning rate
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)
for epoch in range(4000):
    total_loss = 0
    for batch in x_batches:
        optimizer.zero_grad()
        x_hat,z,mu,std = model(batch)
        #print(coordinates_sigmasq)
        #loss = mse_loss(out,batch)
        recon_loss = gaussian_likelihood(x_hat, model.log_scale, batch)
        kl = kl_divergence(z, mu, std)
        elbo = (kl - recon_loss)
        elbo = elbo.mean()
        elbo.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1) #what param to set here? (grads are exploding without cliping)
        optimizer.step()
        total_loss += elbo.item()
    if epoch % 100 == 0:
        print(total_loss) #with 10 epochs 7720 2.36    

In [ ]:
mapping_model = MapToLatent_3().to(cuda0)
mse_loss = nn.MSELoss()
#lr = 0.5 # learning rate
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer_map = torch.optim.Adam(mapping_model.parameters(),weight_decay=1e-5)
for epoch in range(4000):
    total_loss = 0
    for i,batch in enumerate(x_batches):
        optimizer_map.zero_grad()
        with torch.no_grad():
            x_hat,z,mu,std = model(batch)
        #out_mu,out_std = mapping_model(guide_batches[i])
        #out_mu = out_mu.permute((1,0,2))
        #out_std = out_std.permute((1,0,2))
        out = mapping_model(guide_batches[i]).permute((1,0,2))
        #print(out_mu.shape,out_std.shape)
        #loss = -kl_divergence_mapping(z,mu,std,out_mu,out_std).mean()
        #loss = mse_loss(out_mu,mu) + mse_loss(out_std,std)
        
        loss = mse_loss(out,z) 
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(mapping_model.parameters(), 1) #what param to set here? (grads are exploding without cliping)
        optimizer_map.step()
        total_loss += loss.item()
    if epoch % 100 == 0:
        #print(out_mu)
        print(total_loss) #with 10 epochs 7720 2.36    

In [ ]:
p = torch.distributions.Normal(torch.zeros((1,2,16)), torch.ones((1,2,16)))
z = p.rsample()
out = model.decoder(z.to(cuda0))

In [ ]:
vect_start = model(training_data[11:12,:,:])[1]
vect_end = model(training_data[10:11,:,:])[1]
z = torch.lerp(vect_start,vect_end,0.5)

In [ ]:
s = 11 #0.006
stats = model(training_data[s:s+1,:,:])
out = model.decoder(torch.distributions.Normal(stats[2], stats[3]).rsample())
#out = model.decoder(z)
#out = model.decoder(stats[1])
#out = stats[0]
#print(stats[1],stats[2],stats[3])
#out = latent_4
#out = x_batches[0][4:5,:,:]
#out = model.decoder(latent)
#out = training_data[s:s+1,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_conv_4.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv_4.svg') 

In [ ]:
vect_start = model(training_data[11:12,:,:])[1]
vect_end = model(training_data[10:11,:,:])[1]
img_list = []
for i in np.linspace(0,1,21):
    z = torch.lerp(vect_start,vect_end,i)
    out = model.decoder(z)
    real_values = torch.tensor([[0.0,0.0]],device=cuda0)
    for i in range(1,out.shape[2]):
        real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
    segments =[]
    for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
    segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
    figure = Path(*segments)
    fig_attributes = [{'style':f'fill:stroke:rgb({250*p}, 149, 237);fill-opacity:0.2;fill-rule:nonzero;stroke:rgb({250*p}, 149, 237);stroke-width:0.2'}]
    wsvg(figure, attributes = fig_attributes, svg_attributes = svg_attributes, filename='gif_generation.svg')
    cairosvg.svg2png(url="gif_generation.svg",write_to="gif_generation.png")
    #img_list.append(imageio.imread("gif_generation.png"))
    png = Image.open("gif_generation.png")
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    img_list.append(background)
img_list[0].save('transition.gif',append_images=img_list[1:],save_all=True, optimize=False, duration=150, loop=0) #, append_images=img_list[1:]

![SegmentLocal](transition.gif "segment")

In [ ]:
vect_start = model(training_data[13:14,:,:])[1]
vect_end = model(training_data[1:2,:,:])[1]
img_list = []
for i in np.linspace(0,1,21):
    z = torch.lerp(vect_start,vect_end,i)
    out = model.decoder(z)
    real_values = torch.tensor([[0.0,0.0]],device=cuda0)
    for i in range(1,out.shape[2]):
        real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
    segments =[]
    for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
    segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
    wsvg(segments,filename='gif_generation.svg')
    cairosvg.svg2png(url="gif_generation.svg",write_to="gif_generation.png")
    #img_list.append(imageio.imread("gif_generation.png"))
    png = Image.open("gif_generation.png")
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    img_list.append(background)
img_list[0].save('transition_2.gif',append_images=img_list[1:],save_all=True, optimize=False, duration=150, loop=0) #, append_images=img_list[1:]

![SegmentLocal](transition_2.gif "segment")

In [ ]:
vect_start = model(training_data[13:14,:,:])[1]
vect_end = model(training_data[16:17,:,:])[1]
img_list = []
for i in np.linspace(0,1,21):
    z = torch.lerp(vect_start,vect_end,i)
    out = model.decoder(z)
    real_values = torch.tensor([[0.0,0.0]],device=cuda0)
    for i in range(1,out.shape[2]):
        real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
    segments =[]
    for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
    segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
    wsvg(segments,filename='gif_generation.svg')
    cairosvg.svg2png(url="gif_generation.svg",write_to="gif_generation.png")
    #img_list.append(imageio.imread("gif_generation.png"))
    png = Image.open("gif_generation.png")
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    img_list.append(background)
img_list[0].save('transition_3.gif',append_images=img_list[1:],save_all=True, optimize=False, duration=150, loop=0) #, append_images=img_list[1:]

![SegmentLocal](transition_3.gif "segment")

In [ ]:
vect_start = model(training_data[22:23,:,:])[1]
vect_end = model(training_data[21:22,:,:])[1]
img_list = []
for i in np.linspace(0,1,21):
    z = torch.lerp(vect_start,vect_end,i)
    out = model.decoder(z)
    real_values = torch.tensor([[0.0,0.0]],device=cuda0)
    for i in range(1,out.shape[2]):
        real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
    segments =[]
    for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
    segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
    wsvg(segments,filename='gif_generation.svg')
    cairosvg.svg2png(url="gif_generation.svg",write_to="gif_generation.png")
    #img_list.append(imageio.imread("gif_generation.png"))
    png = Image.open("gif_generation.png")
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    img_list.append(background)
img_list[0].save('transition_4.gif',append_images=img_list[1:],save_all=True, optimize=False, duration=150, loop=0) #, append_images=img_list[1:]

![SegmentLocal](transition_4.gif "segment")

In [ ]:
input = input_data_split[s:s+1,:,:]
#z[:,-5][0] = 0
#z[0,-5][1] = 0
input[0,4:5,:] = 0

In [ ]:
s = 60 #0.006
#z = input_data_split[s:s+1,:,:]
#map_mu, map_std = mapping_model(z)
z = mapping_model(input)
#stats = model(training_data[s:s+1,:,:])
#print(z,stats[1])
#out = model.decoder(torch.distributions.Normal(map_mu, map_std).rsample())
out = model.decoder(z)
#out = model.decoder(stats[1])
#out = stats[0]
#print(stats[1],stats[2],stats[3])
#out = latent_4
#out = x_batches[0][4:5,:,:]
#out = model.decoder(latent)
#out = training_data[s:s+1,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_conv_5.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv_5.svg') 

In [ ]:
vect_start = model(training_data[0:1,:,:])[1]
vect_end = model(training_data[15:16,:,:])[1]
img_list = []
for i in np.linspace(0,1,21):
    z = torch.lerp(vect_start,vect_end,i)
    out = model.decoder(z)
    real_values = torch.tensor([[0.0,0.0]],device=cuda0)
    for i in range(1,out.shape[2]):
        real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
    segments =[]
    for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
    segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
    wsvg(segments,filename='gif_generation.svg')
    cairosvg.svg2png(url="gif_generation.svg",write_to="gif_generation.png")
    #img_list.append(imageio.imread("gif_generation.png"))
    png = Image.open("gif_generation.png")
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    img_list.append(background)
img_list[0].save('transition.gif',append_images=img_list[1:],save_all=True, optimize=False, duration=150, loop=0) #, append_images=img_list[1:]

![SegmentLocal](transition_5.gif "segment")

In [ ]:
vect_start = model(training_data[100:101,:,:])[1]
vect_end = model(training_data[15:16,:,:])[1]
img_list = []
for i in np.linspace(0,1,21):
    z = torch.lerp(vect_start,vect_end,i)
    out = model.decoder(z)
    real_values = torch.tensor([[0.0,0.0]],device=cuda0)
    for i in range(1,out.shape[2]):
        real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
    segments =[]
    for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
    segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
    wsvg(segments,filename='gif_generation.svg')
    cairosvg.svg2png(url="gif_generation.svg",write_to="gif_generation.png")
    #img_list.append(imageio.imread("gif_generation.png"))
    png = Image.open("gif_generation.png")
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    img_list.append(background)
img_list[0].save('transition_6.gif',append_images=img_list[1:],save_all=True, optimize=False, duration=150, loop=0) #, append_images=img_list[1:]

![SegmentLocal](transition_6.gif "segment")

In [ ]:
vect_start = model(training_data[111:112,:,:])[1]
vect_end = model(training_data[60:61,:,:])[1]
img_list = []
for i in np.linspace(0,1,21):
    z = torch.lerp(vect_start,vect_end,i)
    out = model.decoder(z)
    real_values = torch.tensor([[0.0,0.0]],device=cuda0)
    for i in range(1,out.shape[2]):
        real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
    segments =[]
    for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
    segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
    wsvg(segments,filename='gif_generation.svg')
    cairosvg.svg2png(url="gif_generation.svg",write_to="gif_generation.png")
    #img_list.append(imageio.imread("gif_generation.png"))
    png = Image.open("gif_generation.png")
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    img_list.append(background)
img_list[0].save('transition_7.gif',append_images=img_list[1:],save_all=True, optimize=False, duration=150, loop=0) #, append_images=img_list[1:]

![SegmentLocal](transition_7.gif "segment")

In [ ]:
s =  6 #0.006
stats = model(training_data[s:s+1,:,:])
out = model.decoder(torch.distributions.Normal(stats[2], stats[3]).rsample())
#out = model.decoder(z)
out = model.decoder(stats[1])
#out = stats[0]
#print(stats[1],stats[2],stats[3])
#out = latent_4
#out = x_batches[0][4:5,:,:]
#out = model.decoder(latent)
#out = training_data[s:s+1,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_conv_6.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv_6.svg') 

In [ ]:
vect_start = model(training_data[111:112,:,:])[1]
vect_end = model(training_data[60:61,:,:])[1]
img_list = []
for p in np.linspace(0,1,21):
    z = torch.lerp(vect_start,vect_end,p)
    out = model.decoder(z)
    real_values = torch.tensor([[0.0,0.0]],device=cuda0)
    for i in range(1,out.shape[2]):
        real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
    segments =[]
    for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
    segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
    figure = Path(*segments)
    svg_attributes = {'viewBox': '-5 -2.5 20 23','height': '600','width': '600'}
    fig_attributes = [{'style':f'fill:none;fill-opacity:1;fill-rule:nonzero;stroke:rgb({250*p}, 149, 237);stroke-width:0.2'}]
    wsvg(figure, attributes = fig_attributes, svg_attributes = svg_attributes, filename='gif_generation.svg')
    cairosvg.svg2png(url="gif_generation.svg",write_to="gif_generation.png")
    #img_list.append(imageio.imread("gif_generation.png"))
    png = Image.open("gif_generation.png")
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    img_list.append(background)
img_list[0].save('transition_11.gif',append_images=img_list[1:],save_all=True, optimize=False, duration=150, loop=0) #, append_images=img_list[1:]

![SegmentLocal](transition_8.gif "segment")

In [ ]:
[{'test':0}]*5

In [ ]:
vect_start = training_data[111:112,:,:]
vect_end = training_data[60:61,:,:]
out = torch.lerp(vect_start,vect_end,0.5)
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
svg_attributes = {'viewBox': '-2 0 20 18','height': '600','width': '600'}
fig_attributes = [{'style':'fill:none;fill-opacity:1;fill-rule:nonzero;stroke:rgb(20, 149, 237);stroke-width:0.1'}]
#fig_attributes[0]['style'] = 'fill:#343542;fill-opacity:1;fill-rule:nonzero;stroke:none;stroke-width:0.0352777'
print(len(segments))
figure = Path(*segments)
wsvg(figure, attributes = fig_attributes, svg_attributes = svg_attributes, filename='test_conv_7.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv_7.svg') 

In [ ]:
s = 111 #0.006
stats = model(training_data[s:s+1,:,:])
#out = model.decoder(torch.distributions.Normal(stats[2], stats[3]).rsample())
#out = model.decoder(z)
#out = model.decoder(stats[1])
out = stats[1]
#print(stats[1],stats[2],stats[3])
#out = latent_4
#out = x_batches[0][4:5,:,:]
#out = model.decoder(latent)
#out = training_data[s:s+1,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
real_values = out[0,:,:].permute((1,0))
fig_attributes = []
segments.append
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
        fig_attributes.append({'style':f'fill:none;stroke:rgba({5*i}, 149, 237,0.5);stroke-width:0.02;stroke-linejoin:bevel'})
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
fig_attributes.append({'style':f'fill:none;stroke:rgba({25*10}, 149, 237,0.5);stroke-width:0.02;stroke-linejoin:bevel'})
wsvg(segments,attributes = fig_attributes,filename='test_conv_13.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv_13.svg') 

In [ ]:
import cairocffi as cairo

In [ ]:
from cairocffi import ImageSurface
from cairosvg.parser import Tree
from cairosvg.surface import SVGSurface


def load_svg(svg: bytes) -> ImageSurface:
    return SVGSurface(Tree(bytestring=svg), None, 1,scale=5).cairo

In [ ]:
surface = load_svg(cairosvg.surface.SVGSurface.convert(url='test_conv_13.svg'))
context = cairo.Context(surface)
context.set_operator(cairo.OPERATOR_DEST_OVER)
context.scale(1973, 2250)
context.set_source_rgba(0,0,0,1)
context.rectangle(0,0,1,1)
context.fill()
surface.write_to_png('pycairo_test_14.png')

In [ ]:
p = torch.distributions.Normal(torch.zeros((1,1,64)), 1)#torch.ones((1,1,64))*0.1)
z = p.rsample()
#out = model.decoder(z.to(cuda0))
#out = model.decoder(stats[1]*1.5)
#vect_start = model(training_data[111:112,:,:])[1]
s = 2
z = model(training_data[s:s+1,:,:])[1]
#vect_end = model(training_data[60:61,:,:])[1]
#z = torch.lerp(vect_start,vect_end,0.5)
out = model.decoder(z.to(cuda0))
#out = model.decoder((vect_end-vect_start)*1.5)
#out = stats[0]
#print(stats[1],stats[2],stats[3])
#out = latent_4
#out = x_batches[0][4:5,:,:]
#out = model.decoder(latent)
#out = training_data[s:s+1,:,:]
real_values = torch.tensor([[0.0,0.0]],device=cuda0)
for i in range(1,out.shape[2]):
    real_values= torch.cat((real_values,torch.unsqueeze(real_values[-1]+out[0,:,i],0)),0)
segments =[]
for i in range(1,real_values.shape[0]):   
        segments.append(Line(complex(real_values[i-1,0],real_values[i-1,1]),complex(real_values[i,0],real_values[i,1])))
segments.append(Line(complex(real_values[i,0],real_values[i,1]),complex()))
wsvg(segments,filename='test_conv_20.svg') #nodes=[complex(),complex(real_values[real_values.shape[0]-1,0],real_values[real_values.shape[0]-1,1])],
SVG('test_conv_20.svg') 

In [ ]:
z

In [ ]:
mu = torch.tensor(np.random.choice([10,50],size=100,p=[0.2,0.8])).float()
torch.abs(torch.distributions.Normal(mu, 10).sample(sample_shape=[1000]).int())

In [ ]:
mu

In [ ]:
torch.abs(torch.distributions.Normal(mu, 10).sample(sample_shape=[1000]).int())